In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV

In [5]:
data = pd.read_csv('data/train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [80]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words='english')),
        ("clf", SVC()),
    ]
)
pipeline

Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', SVC())])

In [81]:
param_grid = {
    "vect__analyzer": ('word', 'char', 'char_wb'),
    "clf__kernel": ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed')
}

In [82]:
grid = GridSearchCV(pipeline, param_grid, scoring='f1')

In [83]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["target"])

In [84]:
grid.fit(X_train, y_train)

/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/lib/python3/dist-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/lib/python3/dist-packages/sklearn/svm/_base.py", line 147, in fit
    raise TypeError("Sparse precomputed kernels are not supported.")
TypeError: Sparse precomputed kernels are not supported.

  warnings.warn("Estimator fit failed. The score on this train-test"


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf', SVC())]),
             param_grid={'clf__kernel': ('linear', 'poly', 'rbf', 'sigmoid',
                                         'precomputed'),
                         'vect__analyzer': ('word', 'char', 'char_wb')},
             scoring='f1')

In [85]:
y_predicted = grid.predict(X_test)

In [86]:
f1_score(y_test, y_predicted)

0.7501647989452868

In [87]:
grid.best_score_, grid.best_params_

(0.7464505974870467, {'clf__kernel': 'linear', 'vect__analyzer': 'word'})

In [88]:
test_data = pd.read_csv('data/test.csv')

In [89]:
target = grid.predict(test_data["text"])

In [90]:
output = pd.DataFrame(data = {
    "id": test_data["id"],
    "target": target
})
output.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [91]:
output.to_csv("submission.csv", index=False)

In [92]:
!kaggle competitions submit -c nlp-getting-started -f "submission.csv" -m "Submission using simple SVC and TF-IDF vectorizer with grid search params (linear, word)"

100%|██████████████████████████████████████| 22.2k/22.2k [00:01<00:00, 21.8kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets